**извлечение признаков из текста на естественном языке**

класстеризатор текстов

частотный анализ с очисткой стоп-слов (TF)

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
806,"""Вежливые люди"" 1939 года. История глазами стариков, переживших шесть властей\n\n5 декабря 2016 в 14:52\n\nTUT.BY\n\nЖурналист и писатель Василий Сарычев уже пятнадцать лет записывает воспоминания...",politics
700,"Гомельские строители утверждают, что их ""кинули"" в Жлобине. Бизнесмен: ""Это ложь и клевета""\n\n9 декабря 2016 в 13:45\n\nЕлена Бычкова / TUT.BY\n\nВ редакцию TUT. BY обратились строители одной из ...",incident


## токенизация и очистка

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
stopwords = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 
    'так', 'его', 'но',  'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или',  'ни', 'быть', 'был', 'него', 'до', 
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 
    'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
    'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 
    'ж', 'тогда', 'кто','этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 
    'этом', 'один', 'почти', 'мой', 'тем','чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем',
    'всех', 'никогда', 'можно', 'при', 'наконец', 'два','об', 'другой', 'хоть', 'после', 
    'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них','какая', 'много', 
    'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
    'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 
    'всегда', 'конечно', 'всю', 'между']

In [8]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in stopwords ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов (предлогов)
    return ' '.join( [ t.strip() for t in tt if t ] )
    

##  CountVectorizer + TF

In [9]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

# ручная очистка
# tf = TfidfVectorizer(preprocessor=preprocessor,use_idf=False,norm=None)
tf = TfidfVectorizer(preprocessor=preprocessor,use_idf=False,norm='l2')

tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f62fd3ab510>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=False, vocabulary=None)

In [10]:
# размер словаря
len(tf.vocabulary_)

37614

## формируем датасеты

In [11]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 37614)

## кластеризируем

In [12]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [13]:
%%time

n_clusters = len(set(data['tag']))
# n_clusters = 5
clust = KMeans(n_clusters=n_clusters, random_state=get_seed() )
clust.fit(X)

CPU times: user 1min 5s, sys: 53.1 ms, total: 1min 5s
Wall time: 55.8 s


In [14]:
# set(clust.labels_)

In [15]:
data['cluster'] = clust.labels_

## тестируем

In [16]:
# размеры кластеров
[ (data['cluster']==c).sum() for c in range(n_clusters) ]

[206, 508, 156, 48, 835, 46, 681, 63, 154, 131, 57, 238, 73]

In [19]:
data[ data['cluster']==1 ].sample(10)

,text,tag,cluster
1841,"Член Совфеда Юрий Воробьев внес в понедельник на рассмотрение нижней палаты Парламента законопроект, вводящий новый порядок госзакупок для Федеральной службы охраны в Госдуму.\n\n«В связи с увелич...",economics,1
2992,"Новая концептуальная автомодельдебютировала в конце августа в Индонезии. Серийная вариация, которая попала на шпионские фотоснимки, обтянута плотной камуфлирующей пленкой. Это мешает рассмотреть э...",auto,1
91,"Новый логотип ""Powered by NetBSD"" The NetBSD Foundation объявило о\nпоявлении нового логотипа ""Powered by NetBSD"".",tech,1
2725,"Разработчики соцсети разрешили пользователям проводить трансляции в прямом эфире.\n\nВладельцы смартфонов iOS и Android после обновления приложения Instagram смогут снимать ""живые"" видео, не перех...",tech,1
190,"Микрочипы составят зрителю личную телепрограмму Похоже, скоро во всём\nмире людям начнуть вшивать RFID-чипы ещё при рождении. По крайней мере,\nна это явно надеется компания TiVo, иначе с чего бы ...",tech,1
1275,"Создан первый язык программирования для ""гопников и реальных пацанов""\n\n9 декабря 2016 в 17:23\n\n42.TUT.BY\n\nСтуденты Петербургского колледжа создали YoptaScript — первый язык для «гопников и р...",tech,1
395,Triplex начинает сотрудничество с Planet Funk 18 ноября Triplex подписал\nсоглашение с продюсерами группы Planet Funk о записи трека для совместного\nпроекта. К работе в неапольской студии Sun Rec...,culture,1
1766,"06:20\n\nНовый член совета директоров Банка России Ольга Полякова, которая возглавляет главное управление по Центральному федеральному округу (ЦФО), будет назначена зампредом ЦБ. Об этом пишет «Ко...",economics,1
346,Партия зеленых: Трехпроцентный барьер мы будем преодолевать с разгона\nВопрос самостоятельного участия Партии зеленых Украины в парламентских\nвыборах или блокирования с другими политическими сила...,politics,1
83,"Настройка печати под FreeBSD Константин Насонов написал статью про настройку\nсистемы сервера печати под FreeBSD, используя CUPS.",tech,1


In [ ]:
# data[ data['cluster']==1 ].sample(10)

In [ ]:
# data[ data['cluster']==2 ].sample(10)

In [ ]:
# data[ data['cluster']==4 ].sample(10)

In [ ]:
# data[ data['cluster']==9 ].sample(10)